# 变分推断

变分推断的主要思想是用简单分布区估计复杂的分布。

假设我们想要估计分布$p(X)$，我们在模型中引入隐变量$Z$，根据贝叶斯公式我们有
$$ ln\,p(X)=ln\,p(X,Z)-ln\,p(Z|X)$$
通过引入一个辅助分布$q(Z)$，上式可以改写为
$$ ln\,p(X)=ln\,\frac{p(X,Z)}{q(Z)}-ln\,\frac{p(Z|X)}{q(Z)}=ln\,p(X,Z)-ln\,q(Z)-ln\,\frac{p(Z|X)}{q(Z)}$$
对等式两边关于$q(Z)$求期望，我们有：
$$ \begin{aligned}ln\,p(X)=\int_z ln\,p(X) q(Z) dz&=\int_z ln\,\frac{p(X,Z)}{q(Z)} q(Z)dz-\int_z ln\,\frac{p(Z|X)}{q(Z)}q(Z)dz\\&=\color{blue}{\underbrace{\int_z ln\,p(X,Z)q(Z)dz-\int_z ln\,q(Z) q(Z)dz}_{\mbox{Evidence Lower Bound}}}+\color{red}{\underbrace{KL(q(Z)||p(Z|X))}_{\mbox{Kullback–Leibler divergence}}}\\&=\mathcal{L}(q)+KL(q||p)\end{aligned}$$

其中第一项叫作证据下界（Evidence Lower Bound，ELOB）,它是关于$q(Z)$的泛函；第二项称为Kullback–Leibler散度，它估计了分布$q(Z)$和$p(Z|X)$的相似度。注意到KL divergence总是非负的，因此我们知道$\mathcal{L}(q)\leq ln p(X)$。当 $q(Z)=p(Z|X)$时，我们有 $KL(q||p)=0$，此时ELOB达到它的最大值,正是$p(X)$的log-likelihood。


我们的目标是在给定数据$X=\{X_1,...,X_n\}$的情况下，使用一个变分分布$q(Z)$去近似隐变量$Z=\{Z_1,...,Z_n\}$的后验分布:
$$ q(Z)\approx p(Z|X)$$
后验概率分布 $p(Z|X)$可能非常复杂，因为给定$X$的情况下，$Z_i$彼此之间通常不是独立的。为了使得近似过程尽量可行,我们假设$Z$可以分解为$M$个互不相交的组(或者说有 $M$个独立的成分):
$$ q(Z)=\prod_{i=1}^M q_i(Z_i)$$
把这个选择代入ELOB,我们有
$$ \underbrace{\int_z \prod_{i=1}^M q_i(Z_i)ln\,p(X,Z)dz}_{part 1}-\underbrace{\int_z \prod_{i=1}^M q_i(Z_i) \sum_{i=1}^M ln\,q_i(Z_i) dz}_{part 2}$$
我们把第一项称为`part 1`，把第二项称为`part 2`，接下来我们分别研究这两项。
### part 1
$$ (part 1)=\int_{Z_1}...\int_{Z_M} \prod_{i=1}^M q_i(Z_i)ln\,p(X,Z)dZ_1, ... ,dZ_M $$
我们把$q_j(Z_j)$从积分中抽取出来并重新排列积分式:
$$ (part 1)=\int_{Z_j} q_j(Z_j)\bigg(\mathop{\int\cdots\int}_{Z_{i\neq j}}ln\,p(X,Z) \prod_{i\neq j}^M  q_i(Z_i)dZ_i\bigg)dZ_j=\int_{Z_j} q_j(Z_j)\mathbb{E}_{i\neq j}[ln\,p(X,Z)]dZ_j$$
### part2
$$\begin{aligned} (part 2)&=\int_z \prod_{i=1}^M q_i(Z_i) \sum_{i=1}^M ln\,q_i(Z_i) dz\\&=\int_{Z_1}...\int_{Z_M} \sum_{i=1}^M ln\,q_i(Z_i) \prod_{j=1}^M q_j(Z_j)dZ_j\\&=\int_{Z_1}...\int_{Z_M} \sum_{i=1}^M ln\,q_i(Z_i) (\prod_{j\neq i}^M q_j(Z_j)dZ_j)dZ_i\\&=\int_{Z_i}\sum_{i=1}^M ln\,q_i(Z_i)\bigg(\mathop{\int\cdots\int}_{Z_{j\neq i}}\prod_{j\neq i}^M q_j(Z_j)dZ_j\bigg)dZ_i\\&=\int_{Z_i}\sum_{i=1}^M q_i(Z_i) ln\,(q_i(Z_i) )dZ_i\\&=\sum_{i=1}^M \int_{Z_i}q_i(Z_i) ln\,(q_i(Z_i) )dZ_i \end{aligned}$$
对于特定的 $q_j(Z_j)$,和的剩余部分可以视为常数
$$ (part2)=\int_{Z_j}q_j(Z_j) ln\,(q_j(Z_j) )dZ_j+const$$
然后
$$\begin{aligned}\mathcal{L}(q)&=part1-part2=\int_{Z_j} q_j(Z_j)\mathbb{E}_{i\neq j}[ln\,p(X,Z)]dZ_j-\int_{Z_j}q_j(Z_j) ln\,(q_j(Z_j) )dZ_j+const\\&=\int_{Z_j} q_j(Z_j)\mathbb{E}_{i\neq j}[ln\,p(X,Z)]dZ_j+const_1\int_{Z_j}  q_j(Z_j)dZ_j-\int_{Z_j}q_j(Z_j) ln\,(q_j(Z_j) )dZ_j+const_2\\&=\int_{Z_j} q_j(Z_j)(\mathbb{E}_{i\neq j}[ln\,p(X,Z)]+const_1)dZ_j-\int_{Z_j}q_j(Z_j) ln\,(q_j(Z_j) )dZ_j+const_2\end{aligned}$$
其中 $const_1+const_2=const$
设$ln\,\tilde{p}_j(X,Z_j)=\mathbb{E}_{i\neq j}[ln\,p(X,Z)]+const_1$,接着我们有
$$\tilde{p}_j(X,Z_j)=\exp(\mathbb{E}_{i\neq j}[ln\,p(X,Z)]+const_1)=\exp(\mathbb{E}_{i\neq j}[ln\,p(X,Z)])\exp(const_1)$$
其中常数项$\exp(const_1)$的作用是归一化。



如果我们将$\mathcal{L}(q)$视为$q_j(Z_j)$的泛函，同时将剩余的分布$q_{i\neq j}(Z_i)$ 固定,我们可以将 ELOB表示为
$$\mathcal{L}(q_j)=\int_{Z_j} q_j(Z_j)ln\,\frac{\tilde{p}_j(X,Z_j)}{q_j(Z_j)}dZ_j+const_2$$
这等价于最小化$-KL( q_j(Z_j)||\exp(\mathbb{E}_{i\neq j}[ln\,p(X,Z)]+const_1))$
因此最大化 ELOB等价于最小化这个特殊的 KL divergence,最优分布$q^*_j(Z_j)$ 满足
$$ ln\,(q^*_j(Z_j))=\mathbb{E}_{i\neq j}[ln\,p(X,Z)]+const_1$$
其中 $const_1$ 作用是归一化分布 $q^*_j(Z_j)$.如果我们在等式两边同时取指数exp则有
$$ q^*_j(Z_j)=\exp(\mathbb{E}_{i\neq j}[ln\,p(X,Z)]+const_1)$$
由
$$\int q^*_j(Z_j) dZ_j=\int \exp(\mathbb{E}_{i\neq j}[ln\,p(X,Z)]+const_1) dZ_j=1\to \exp(const_1)=\frac{1}{\int\exp(\mathbb{E}_{i\neq j}[ln\,p(X,Z)]dZ_j}$$
我们可以得到变分推断的更新式
$$ q^*_j(Z_j)=\frac{\exp(\mathbb{E}_{i\neq j}[ln\,p(X,Z)])}{\int\exp(\mathbb{E}_{i\neq j}[ln\,p(X,Z)]dZ_j}$$
